# Concatenative Paradigm

Tacit Programming Approach (Concatenative)

https://www.wikiwand.com/en/Tacit_programming
https://www.wikiwand.com/en/Concatenative_programming_language

To achieve this:

- Functions do not use explicit parameters
- Branchless

## Auxiliary Functions

In [24]:
import operator
import functools
from typing import Callable, TypeVar

T = TypeVar("T")
R = TypeVar("R")

T1 = TypeVar("T1")
T2 = TypeVar("T2")
T3 = TypeVar("T3")


def compose(
    function_outer: Callable[[T1], T2], function_inner: Callable[[T3], T1]
) -> Callable[[T3], T2]:
    def composed_function(*args: T3) -> T2:
        result = function_inner(*args)
        return function_outer(result)

    return composed_function


def thrush(value: T) -> Callable[[Callable[[T], R]], R]:
    def thrush_step(function: Callable[[T], R]) -> R:
        return function(value)

    return thrush_step


def iterate(function: Callable[[T | R], R]) -> Callable[[int], Callable[[T], R | T]]:
    def iterate_step(times: int) -> Callable[[T], R | T]:
        def inner_step(x: R | T) -> R | T:
            return functools.reduce(lambda x, _: function(x), range(times), x)

        return inner_step

    return iterate_step


def curry(
    function: Callable[[T], R]
) -> Callable[[T], Callable[[T], R] | functools.partial[R]]:
    def curry_step(*args: T) -> Callable[[T], R] | functools.partial[R]:
        result = function
        for arg in args:
            result = functools.partial(function, arg)
        return result

    return curry_step


def uncurry(function: Callable[[T], Callable[[T], R] | R]) -> Callable[[T], R]:
    def uncurry_step(*args: T) -> R:
        result = function
        for arg in args:
            result = result(arg)
        return result

    return uncurry_step


## Predecessor and Successor

In [25]:
predecessor = functools.partial(operator.add, -1)

assert predecessor(1) == 0
assert predecessor(10) == 9


In [26]:
successor = functools.partial(operator.add, 1)

assert successor(0) == 1
assert successor(10) == 11


## Addition

In [27]:
addition = uncurry(iterate(successor))

assert addition(1, 0) == 1
assert addition(0, 0) == 0
assert addition(0, 1) == 1
assert addition(10, 10) == 20


## Multiplication

In [28]:
multiplication = compose(thrush(0), uncurry(compose(iterate, curry(addition))))

assert multiplication(0, 0) == 0
assert multiplication(2, 0) == 0
assert multiplication(0, 2) == 0
assert multiplication(10, 10) == 100


## Exponentiation

In [29]:
exponentiation = compose(thrush(1), uncurry(compose(iterate, curry(multiplication))))

assert exponentiation(1, 0) == 1
assert exponentiation(0, 1) == 0
assert exponentiation(3, 3) == 27
